In [1]:
%run ontologyPackage/ontologySTATanalysis.ipynb

In [2]:
import pandas as pd 
import numpy as np
from numpy import random, array
import time 
# from multiprocessing import Pool
import rpy2.robjects as robjects

import os
from goatools.obo_parser import GODag

pd.set_option('display.max_colwidth', None) # Values in columns won't be shortned | None over -1
pd.set_option('chained_assignment',None) # Disabling chained assignments 

geneColID = ["chromosome","source","type","start","end","score","strand","phase","gene_symbol","gene_ensID","length","entrezid"]
geneAnnotationDF = pd.read_csv('entrez_id/geneAnnotationsDF_Selected_entrezID.csv', sep=',', comment='#', low_memory=False, header=0, names=geneColID)
chromosomeColID = ['chromosome','source','type','start','end','score','strand','phase']
chromosomesDF = pd.read_csv('chromosomesDF.csv', sep=',', comment='#', low_memory=False, header=0, names=chromosomeColID)

In [3]:
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests as padjust
from statistics import median, stdev

In [4]:
# GO Term information
if not os.path.exists('go-basic.obo'):
    !wget http://geneontology.org/ontology/go-basic.obo
goDATA = GODag('go-basic.obo', optional_attrs=['relationship'])

go-basic.obo: fmt(1.2) rel(2019-07-01) 47,413 GO Terms; optional_attrs(relationship)


In [5]:
#Dropping unneccesary stuff and resetting index from 0
# geneAnnotationDF = geneAnnotationDF[geneAnnotationDF.gene_symbol != 'CTD-2207O23.3']
geneAnnotationDF = geneAnnotationDF.sort_values(by=['chromosome', 'start'])
chromosomesDF = chromosomesDF.sort_values(by=['chromosome'])
chromosomesDF = chromosomesDF.reset_index()
geneAnnotationDF = geneAnnotationDF.reset_index()
chromosomesDF = chromosomesDF.drop(columns=['score', 'strand', 'phase', 'index'])
geneAnnotationDF = geneAnnotationDF.drop(columns=['score', 'phase', 'index'])

In [6]:
# Making a list of DataFrames to be used in addwindow tss so that if - gene the start is its end and its end is its start 
u = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','MT','X','Y'] # Chromosome ids 
geneDFL = []
for c in u:
    geneDF = geneAnnotationDF.loc[geneAnnotationDF['chromosome'] == c].copy()
    for i, row in geneDF.iterrows():
        if row['strand'] == '-':
            start = row['end']
            end = row['start']
            geneDF.at[i, 'start'] = start
            geneDF.at[i, 'end'] = end
    geneDF = geneDF.sort_values(by=['start'])
    geneDF = geneDF.reset_index()
    geneDF = geneDF.drop(columns=['index'])
    geneDFL.append(geneDF)

In [7]:
# Making a list of DataFrames to be used in future functions in var geneDFL
u = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','MT','X','Y']
geneDFLBODY = []
for cc in u:
    geneDFbod = geneAnnotationDF.loc[geneAnnotationDF['chromosome'] == cc].copy()
    geneDFbod = geneDFbod.sort_values(by=['start'])
    geneDFbod = geneDFbod.reset_index()
    geneDFbod = geneDFbod.drop(columns=['index'])
    geneDFLBODY.append(geneDFbod)

In [8]:
# probability that a randomly chosen gene is in a certain chromosome. Ordered from 1 - MT,X,Y | As shown chromosome 1 has highest prob of selection
probabilityL = [0.08069467597786323, 0.07850260775517634, 0.06427388269957329, 0.06165457287524136, 0.05884231002379223, 0.05536363751707386, 0.05164908592141782, 0.0470440371698401, 0.044858119022639725, 0.043367989830914035, 0.043785860460049814, 0.04319875643982657, 0.037069107410936775, 0.034696265410969616, 0.033058580434273406, 0.029281523923645837, 0.026986378244674356, 0.026051531764496164, 0.018999829086634144, 0.02088839915494138, 0.015140187376094325, 0.016471877735809576, 5.370214538878023e-06, 0.05057780526426391, 0.017537607961181506]

In [9]:
# Generates random sites
dl = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25'] # 23, 24, 25
chrD = {'1': [1, 248956422],'2': [1, 242193529],'3': [1, 198295559],'4': [1, 190214555],'5': [1, 181538259],'6': [1, 170805979],'7': [1, 159345973],'8': [1, 145138636],'9': [1, 138394717],'10': [1, 133797422],'11': [1, 135086622],'12': [1, 133275309],'13': [1, 114364328],'14': [1, 107043718],'15': [1, 101991189],'16': [1, 90338345],'17': [1, 83257441],'18': [1, 80373285],'19': [1, 58617616],'20': [1, 64444167],'21': [1, 46709983],'22': [1, 50818468],'23': [1, 16569],'24': [1, 156040895],'25': [2781480, 56887902]}
def nRandSites(data):
    nSite = data[0]
    rand = data[1] 
    sitesbyC =[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]

    random.seed(rand)
        
    for site in range(nSite):
        chrN = random.choice(dl, p=probabilityL)
        randnum = round(random.uniform(chrD[chrN][0], chrD[chrN][1]))
        sitesbyC[int(chrN) - 1].append(randnum)
    for l in sitesbyC:
        l.sort()
    return array([array(l) for l in sitesbyC])

def nRandSitesSim(nSite, nSim):
    totalsites = [nSite] * nSim
    chunks = []
    for i in totalsites:
        chunks.append([i])
    for l in chunks:
        l.append(random.randint(100000))
    
    pool = Pool(5)
    result = pool.map(nRandSites, chunks)
    pool.close()
    return(result) 

In [10]:
##### Adds midpoint in gene and upper and lower bound values to geneDFLtss based on size of window or nearby genes. Adding around geneTSS.
def addWindowTSS(window):
    chrD = {'1': [1, 248956422],'2': [1, 242193529],'3': [1, 198295559],'4': [1, 190214555],'5': [1, 181538259],'6': [1, 170805979],'7': [1, 159345973],'8': [1, 145138636],'9': [1, 138394717],'10': [1, 133797422],'11': [1, 135086622],'12': [1, 133275309],'13': [1, 114364328],'14': [1, 107043718],'15': [1, 101991189],'16': [1, 90338345],'17': [1, 83257441],'18': [1, 80373285],'19': [1, 58617616],'20': [1, 64444167],'21': [1, 46709983],'22': [1, 50818468],'23': [1, 16569],'24': [1, 156040895],'25': [2781480, 56887902]} 
    geneWindowDFL = []
    for i in range(len(geneDFL)):
        df = geneDFL[i].copy()
        geneWindowDFL.append(df)
    outList = []
    chrIDNUM = 0
    for dfG in geneWindowDFL: # ordered by chromosome 1- MT,X,Y
        chrIDNUM += 1 
        dfIndexMax = len(dfG) - 1   
        for i, row in dfG.iterrows():
            start = row['start']
            
            dfG.at[i, 'midBody'] = (((row['length'] / 2)) + start) # Middle of gene body 
            if i == 0 or i == dfIndexMax: # Case for start and end genes
                if i == 0:
                    dfG.at[i, 'lowB'] = max(start - window, chrD[str(chrIDNUM)][0]) # chrD[chrIDNUM][0] = start val of the chromosome
                    dfG.at[i, 'upperB'] = min(start + window, ((dfG.iat[i+1, 3] - start) / 2) + start)
                else: # index max case 
                    dfG.at[i, 'lowB'] = max(start - window, start - ((start - dfG.iat[i-1, 3]) / 2)) # dfG.iat[i-1, 3] = start of gene below
                    dfG.at[i, 'upperB'] = min(start + window, chrD[str(chrIDNUM)][1]) # chrD[chrIDNUM][1] = end val of chromosome 
            else: # i > 0 or i < len(geneDF) - 1 
                dfG.at[i, 'lowB'] = max(start - window, start - ((start - dfG.iat[i-1, 3]) / 2)) # dfG.iat[i-1, 3] = start of gene below
                dfG.at[i, 'upperB'] = min(start + window, ((dfG.iat[i+1, 3] - start) / 2) + start) # Start of the gene above position [i,3]
        dfG['midBody'] = dfG['midBody'].astype(int)
        dfG['lowB'] = dfG['lowB'].astype(int)
        dfG['upperB'] = dfG['upperB'].astype(int)
#         dfG['domainLEN'] = dfG['upperB'] - dfG['lowB']
        LBandUB = []
        LBandUB.append(dfG.lowB.values)
        LBandUB.append(dfG.upperB.values)
        outList.append(LBandUB)
    
    return(outList)

In [11]:
##### Adds midpoint in gene and upper and lower bound values to geneDFLtss based on size of window or nearby genes. Adding around geneTSS.
def addWindowTSS20(window):
    chrD = {'1': [1, 248956422],'2': [1, 242193529],'3': [1, 198295559],'4': [1, 190214555],'5': [1, 181538259],'6': [1, 170805979],'7': [1, 159345973],'8': [1, 145138636],'9': [1, 138394717],'10': [1, 133797422],'11': [1, 135086622],'12': [1, 133275309],'13': [1, 114364328],'14': [1, 107043718],'15': [1, 101991189],'16': [1, 90338345],'17': [1, 83257441],'18': [1, 80373285],'19': [1, 58617616],'20': [1, 64444167],'21': [1, 46709983],'22': [1, 50818468],'23': [1, 16569],'24': [1, 156040895],'25': [2781480, 56887902]} 
    geneWindowDFL = []
    for i in range(len(geneDFL)):
        df = geneDFL[i].copy()
        geneWindowDFL.append(df)
    chrIDNUM = 0
    for dfG in geneWindowDFL: # ordered by chromosome 1- MT,X,Y
        chrIDNUM += 1 
        dfIndexMax = len(dfG) - 1   
        for i, row in dfG.iterrows():
            start = row['start']
            
            dfG.at[i, 'midBody'] = (((row['length'] / 2)) + start) # Middle of gene body 
            if i == 0 or i == dfIndexMax: # Case for start and end genes
                if i == 0:
                    dfG.at[i, 'lowB'] = max(start - window, chrD[str(chrIDNUM)][0]) # chrD[chrIDNUM][0] = start val of the chromosome
                    dfG.at[i, 'upperB'] = min(start + window, ((dfG.iat[i+1, 3] - start) / 2) + start)
                else: # index max case 
                    dfG.at[i, 'lowB'] = max(start - window, start - ((start - dfG.iat[i-1, 3]) / 2)) # dfG.iat[i-1, 3] = start of gene below
                    dfG.at[i, 'upperB'] = min(start + window, chrD[str(chrIDNUM)][1]) # chrD[chrIDNUM][1] = end val of chromosome 
            else: # i > 0 or i < len(geneDF) - 1 
                dfG.at[i, 'lowB'] = max(start - window, start - ((start - dfG.iat[i-1, 3]) / 2)) # dfG.iat[i-1, 3] = start of gene below
                dfG.at[i, 'upperB'] = min(start + window, ((dfG.iat[i+1, 3] - start) / 2) + start) # Start of the gene above position [i,3]
        dfG['midBody'] = dfG['midBody'].astype(int)
        dfG['lowB'] = dfG['lowB'].astype(int)
        dfG['upperB'] = dfG['upperB'].astype(int)
        dfG['domainLEN'] = dfG['upperB'] - dfG['lowB']
    
    return(geneWindowDFL)

In [12]:
# Adds midpoint in gene and upper and lower bound values to geneDFL based on size of window or nearby genes. Adding around geneBODY.
def addWindowBODY(window):
    chrD = {'1': [1, 248956422],'2': [1, 242193529],'3': [1, 198295559],'4': [1, 190214555],'5': [1, 181538259],'6': [1, 170805979],'7': [1, 159345973],'8': [1, 145138636],'9': [1, 138394717],'10': [1, 133797422],'11': [1, 135086622],'12': [1, 133275309],'13': [1, 114364328],'14': [1, 107043718],'15': [1, 101991189],'16': [1, 90338345],'17': [1, 83257441],'18': [1, 80373285],'19': [1, 58617616],'20': [1, 64444167],'21': [1, 46709983],'22': [1, 50818468],'23': [1, 16569],'24': [1, 156040895],'25': [2781480, 56887902]} 
    chrIDNUM = 0
    geneWindowDFL = []
    for i in range(len(geneDFLBODY)):
        df = geneDFLBODY[i].copy()
        geneWindowDFL.append(df)
    
    outList = []
    for dfG in geneWindowDFL: # ordered by chromosome 1- MT,X,Y
        chrIDNUM += 1 
        dfIndexMax = len(dfG) - 1         
        for i, row in dfG.iterrows():
            start = row['start']
            end = row['end']
            dfG.at[i, 'midBody'] = (((row['length'] / 2)) + row['start']) # Middle of gene body 
            
            if i == 0 or i == dfIndexMax: # Case for start and end sites
                if i == 0:
                    dfG.at[i, 'lowB'] = max(start - window, chrD[str(chrIDNUM)][0]) # end of the gene below position
                    dfG.at[i, 'upperB'] = min(end + window, ((dfG.iat[i+1, 3] - end) / 2) + end)
                else: # index max case 
                    dfG.at[i, 'lowB'] = max(start - window, start - (abs(start - dfG.iat[i-1, 4]) / 2)) 
                    dfG.at[i, 'upperB'] = min(row['end'] + window, chrD[str(chrIDNUM)][1])
            else: # i > 0 or i < len(geneDF) - 1 
                dfG.at[i, 'lowB'] = max(start - window, start - (abs(start - dfG.iat[i-1, 4]) / 2)) # end of the gene below position [i,4]
                dfG.at[i, 'upperB'] = min(end + window, ((dfG.iat[i+1, 3] - end) / 2) + end) # Start of the gene above position [i,3]
        dfG['midBody'] = dfG['midBody'].astype(int)
        dfG['lowB'] = dfG['lowB'].astype(int)
        dfG['upperB'] = dfG['upperB'].astype(int)
#         dfG['domainLEN'] = dfG['upperB'] - dfG['lowB']
        LBandUB = []
        LBandUB.append(dfG.lowB.values)
        LBandUB.append(dfG.upperB.values)
        outList.append(LBandUB)
    
    return(outList)

In [13]:
# Adds midpoint in gene and upper and lower bound values to geneDFL based on size of window or nearby genes. Adding around geneBODY.
def addWindowBODY20(window):
    chrD = {'1': [1, 248956422],'2': [1, 242193529],'3': [1, 198295559],'4': [1, 190214555],'5': [1, 181538259],'6': [1, 170805979],'7': [1, 159345973],'8': [1, 145138636],'9': [1, 138394717],'10': [1, 133797422],'11': [1, 135086622],'12': [1, 133275309],'13': [1, 114364328],'14': [1, 107043718],'15': [1, 101991189],'16': [1, 90338345],'17': [1, 83257441],'18': [1, 80373285],'19': [1, 58617616],'20': [1, 64444167],'21': [1, 46709983],'22': [1, 50818468],'23': [1, 16569],'24': [1, 156040895],'25': [2781480, 56887902]} 
    chrIDNUM = 0
    geneWindowDFL = []
    for i in range(len(geneDFLBODY)):
        df = geneDFLBODY[i].copy()
        geneWindowDFL.append(df)
    
    for dfG in geneWindowDFL: # ordered by chromosome 1- MT,X,Y
        chrIDNUM += 1 
        dfIndexMax = len(dfG) - 1         
        for i, row in dfG.iterrows():
            start = row['start']
            end = row['end']
            
            dfG.at[i, 'midBody'] = (((row['length'] / 2)) + row['start']) # Middle of gene body 
            
            if i == 0 or i == dfIndexMax: # Case for start and end sites
                if i == 0:
                    dfG.at[i, 'lowB'] = max(start - window, chrD[str(chrIDNUM)][0]) # end of the gene below position
                    dfG.at[i, 'upperB'] = min(end + window, ((dfG.iat[i+1, 3] - end) / 2) + end)
                else: # index max case 
                    dfG.at[i, 'lowB'] = max(start - window, start - (abs(start - dfG.iat[i-1, 4]) / 2)) 
                    dfG.at[i, 'upperB'] = min(row['end'] + window, chrD[str(chrIDNUM)][1])
            else: # i > 0 or i < len(geneDF) - 1 
                dfG.at[i, 'lowB'] = max(start - window, start - (abs(start - dfG.iat[i-1, 4]) / 2)) # end of the gene below position [i,4]
                dfG.at[i, 'upperB'] = min(end + window, ((dfG.iat[i+1, 3] - end) / 2) + end) # Start of the gene above position [i,3]
        dfG['midBody'] = dfG['midBody'].astype(int)
        dfG['lowB'] = dfG['lowB'].astype(int)
        dfG['upperB'] = dfG['upperB'].astype(int)
        dfG['domainLEN'] = dfG['upperB'] - dfG['lowB']
    
    return(geneWindowDFL)

In [14]:
def findSMLdomain(windowDF):
    windowLens = windowDF
    allVALS = []
    for df in windowLens:
        values = list(df.domainLEN.values)
        allVALS += values
    allVALS = sorted(allVALS)
    cutoff = int(len(allVALS) / 3)
    small = allVALS[:cutoff][-1]
    medium = allVALS[cutoff:cutoff*2][-1]
    return [small, medium]

In [15]:
# Creating list of all the entrezids 
entrezIDLtss = []
for geneDF in geneDFL:
    entrezIDLtss.append(geneDF.entrezid.values)

In [16]:
entrezIDLbody = []
for geneDF in geneDFLBODY:
    entrezIDLbody.append(geneDF.entrezid.values)

In [100]:
# CLOSEST Genes potentionally regulated by sites in window with respect to the genes body. k = 1 
def geneReadSites(bsL, geneWindow, method='TSS'): # geneWindow = windows lists
    #     geneWindow = addWindowBODY(window)
    chromosomeI = 0    # 0 == chromosome group 1          
    mappedEntrezG = [] # Output list of entrez ids 
    
    if method == 'TSS':
        entrezIDL = entrezIDLtss.copy()
    elif method == 'BODY':
        entrezIDL = entrezIDLbody.copy()
    elif method != 'TSS'and method != 'BODY':
        return 'method must be TSS or BODY'
    
    for bounds in geneWindow:
        lowB = bounds[0]
        upperB = bounds[1]
        geneIDS = entrezIDL[chromosomeI]
        sitesL = bsL[chromosomeI]
        
        for site in sitesL:
            for i in range(len(lowB)):
                if site < lowB[i]:
                    lowB = lowB[i:] # Getting rid of the gene windows because it has been mapped. Genes at a lower positions are removed casuse they are small
                    upperB = upperB[i:]
                    geneIDS = geneIDS[i:]
                    break
                if lowB[i] <= site and upperB[i] > site:
                    mappedEntrezG.append(geneIDS[i])
#                     lowB = lowB[i+1:] # Getting rid of the gene windows because it has been mapped. Genes at a lower positions are removed casuse they are small
#                     upperB = upperB[i+1:]
#                     geneIDS = geneIDS[i+1:]
#                     break # Done with the current site so break 
        chromosomeI += 1   
    return mappedEntrezG

In [101]:
# Given a list of lists with go term and its odds-ratio,p-val return a list of just GO terms 
def getGOfromAnalysis(goAnalysis):
    goTermL = []
    for key in goAnalysis:
        goTermL.append(key)
    return goTermL

In [102]:
# Given original analysis and simulated analysis. Adds count to go term in original list if the random go term has a lower p-val and higher odds ratio 
def compareGOAnalysis(origAnalysis, counters):
    for k in counters.keys():
        if origAnalysis[k][2] != 'NA':
            if (counters[k][0] < origAnalysis[k][2]): #  (simAnalysis[k][0] > origAnalysis[k][0]) and 
                origAnalysis[k][5] += 1
        if origAnalysis[k][3] != 'NA':
            if (counters[k][1] < origAnalysis[k][3]): 
                origAnalysis[k][6] += 1
        if origAnalysis[k][4] != 'NA':
            if (counters[k][2] < origAnalysis[k][4]): 
                origAnalysis[k][7] += 1
    return origAnalysis 

In [103]:
# converts analysis to list format and divides by nSim to get refabs p value
def convertAnalysistoFormat(analysis, nSim):
    analist = []
    for k in analysis.keys():
        val = analysis[k]
        outval = val[:2]
        app = val[5:]
        pval = []
        for i in app:
            p = (i + 1) / (nSim + 1)
            pval.append(p)
        nas = val[2:5]
        for i in range(3):
            if nas[i] == 'NA':
                pval[i] = 'NA'
        outval += pval
        analist.append([k, outval])
    return analist

In [104]:
r = robjects.r

In [105]:
# Combines the three different p-vals together \ Format: pvals = [x,y,z]
def reFABScalc(pvals):    
    r['source']("SMLmetapADJ.R")
    return r("reFABSp(c(" + str(pvals)[1:-1] + "))")   

In [106]:
# Combines the S M & L, pvals for each GO term into one from the adaptiveEnrichmentAnalysis() function || adaptiveEnrichmentAnalysis(randsites, 100000, method='TSS')
# Replaces 1.0 with 0.9999999 & removes 'NA' for genes without s/m/l
def reFABSlistC(enrichmentA):
    r['source']("SMLmetapADJ.R")
    goTerms = []
    pvals = []
    for GO in enrichmentA:
        pvals.append([GO[1][1],GO[2][1],GO[3][1]])
        goTerms.append(GO[0])
    adjPval = []
    for l in pvals:
        newL = []
        for ind in range(len(l)):
            if type(l[ind]) != str:
                if l[ind] > 0.9999999:
                    newL.append(0.9999999)
                else:
                    newL.append(l[ind])
        adjPval.append(newL)
#     return adjPval
    
    stringreturn = "reFABSp(list("
    for ind in range(len(adjPval)):
        stringreturn  += "c("+ str(adjPval[ind])[1:-1] +"), "   
    output = []
    data = r(stringreturn[:-2] + "))") 
    for goID in range(len(goTerms)):
        output.append([goTerms[goID], data[goID]])
    return output



In [31]:
testing=adaptiveEnrichmentAnalysis(randsites, 10000, method='TSS')

In [44]:
combined = reFABSlistC(testing)

In [139]:
File = pd.read_csv('polyenrich/SmLgzFiles/Converted Data-hg38/largeDatasetConverted.csv')

In [140]:
chrlabel = ['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10','chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chr21','chr22','chrMT','chrX','chrY']




In [141]:
File

,Unnamed: 0,Chromo,Start,End,Site
0,0,chr1,225474969,225475318,225475143
1,1,chr16,55465105,55465447,55465276
2,2,chr10,64041215,64041557,64041386
3,3,chr5,139262670,139262962,139262816
4,4,chr17,43760373,43760708,43760540
...,...,...,...,...,...
75630,75638,chr8,80890024,80890264,80890144
75631,75639,chr10,74891915,74892155,74892035
75632,75640,chr6,112148183,112148423,112148303
75633,75641,chr2,214326179,214326419,214326299


In [142]:
len(File[File.Chromo.isin(chrlabel)])

75635

In [84]:
File.to_csv('polyenrich/SmLgzFiles/Converted Data-hg38/largeDatasetConverted.csv')

In [143]:
chrList = list(File['Chromo'].values)
SiteLoc = list(File['Site'].values)

In [144]:
len(SiteLoc)

75635

In [145]:
FileData = []
for item in range(len(SiteLoc)):
    FileData.append([chrList[item], SiteLoc[item]])

In [146]:
len(FileData)

75635

In [147]:
FileSites = []
for x in range(25):
    FileSites.append([])

In [148]:
for item in range(len(FileData)):
    FileSites[chrlabel.index(FileData[item][0])].append(FileData[item][1])

        

In [149]:
c = 0 
for i in FileSites:
    c+=len(i)
c

75635

In [150]:
len(FileSites)

25

In [177]:
testSite = nRandSitesSim(75680, 1)

In [179]:
len(testSite[0])

25

In [151]:
mappedFile = geneReadSites(FileSites, addWindowTSS(10000), method='TSS')

In [153]:
len(mappedFile)

63

In [115]:
window = addWindowTSS20(10000)

In [134]:
geneAnnotationDF[geneAnnotationDF['entrezid']==79894]

,chromosome,source,type,start,end,strand,gene_symbol,gene_ensID,length,entrezid
2201,1,ensembl_havana,gene,248838210,248849517,+,ZNF672,ENSG00000171161,11308,79894


In [135]:
window[0][window[0]['entrezid']==79894]

,chromosome,source,type,start,end,strand,gene_symbol,gene_ensID,length,entrezid,midBody,lowB,upperB,domainLEN
2201,1,ensembl_havana,gene,248838210,248849517,+,ZNF672,ENSG00000171161,11308,79894,248843864,248832421,248848210,15789


In [157]:
site = 204162667
chro = 15
lowB = window[chro]['lowB'].values
upperB = window[chro]['upperB'].values
for i in range(len(lowB)):
    if lowB[i] <= site and upperB[i] > site:
        print(lowB[i], upperB[i], chro)

    

In [131]:
26155 in mappedFile

False

In [109]:
# Gets dinfo for GO terms. (AVG domain, Median domain length, Domain length stdev, # genes, goname, go namespace) || TSS only ATM 
def getgoINFO(data, window, type='TSS'):
    output = []
    pos = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','MT','X','Y']
    
    if type == 'TSS':
        windDF = addWindowTSS20(window)
    elif type == 'BODY':
        windDF = addWindowBODY20(window)
    else:
        raise InputError("Incorrect option")
    
    for item in data:
        add = []
        add.append(item)
        try:
            genes = go2gene[item]
        except KeyError:
            output.append(add)
            continue 
        
        geneDom = []
        for gene in genes:
            try:
                chromo = geneAnnotationDF.loc[geneAnnotationDF['entrezid'] == gene].chromosome.values[0]   
            except IndexError:
                continue
            
            try:
                geneDomain = int(windDF[pos.index(chromo)].loc[windDF[pos.index(chromo)]['entrezid'] == gene].domainLEN.values[0])
            except ValueError:
                continue
            
            geneDom.append(geneDomain)
            
        if len(geneDom) == 0:
            output.append(add)
            continue 
            
        domainAVG = sum(geneDom)/len(geneDom)
        
        medianValue = median(geneDom)
        if len(geneDom) > 1:
            standardDev = stdev(geneDom)
        else:
            standardDev = 0
        
        add.append(domainAVG)
        add.append(medianValue)  
        add.append(standardDev)  
        add.append(len(go2gene[item]))
        try:
            add.append(goinfo.nodes[item]['name'])
            add.append(goinfo.nodes[item]['namespace'])
        except KeyError:
            add.append('NA')
            add.append('NA')
            
        output.append(add)
        
    return output
        
            
    

In [110]:
# Compares original analysis with simulated analysis for nSim
def simulation(GOlist, geneL, nSim, nSites, origAnalysis, method, SMLcutoff, entrezDomains): # GOList is the new go list that we analyze through since we don't want whole GO list, geneL is windowDF
    sitesL = nRandSitesSim(nSites, nSim)
    outputAnalysis = origAnalysis # We will keep updating this dictionary and return when all sims are done 
    
    for sim in range(nSim):
        mapped = geneReadSites(sitesL[sim], geneL, method)
        counters = SMLcounterFAST(GOlist, mapped, SMLcutoff, entrezDomains)
        outputAnalysis = compareGOAnalysis(outputAnalysis, counters)
        
    return sorted(convertAnalysistoFormat(outputAnalysis, nSim), key = lambda x: x[1][1])

In [111]:
# First run that sets up for the simulation
def firstRun(userSitesL, window, method='TSS', simN=100): # userSites must be ordered by chromosome number (List of lists) # Default simN = 10000
    if method == 'TSS':
        geneL = addWindowTSS(window)
        windowDF = addWindowTSS20(window)
    elif method == 'BODY': 
        geneL = addWindowBODY(window)
        windowDF = addWindowBODY20(window)
    else:
        return 'Only TSS or BODY method allowed!'
    
    geneClassification = findSMLdomain(windowDF)
    
    mappedGenes = geneReadSites(userSitesL, geneL, method)
    
    numMapped = len(mappedGenes)
    numsites = 0
    for l in userSitesL:
        numsites += len(l)
    
    entrezDomains = getEntrezDomain(windowDF)
#     return mappedGenes, entrezDomains
    goAnalysis = conductAnalysisFIRST(mappedGenes, geneClassification, entrezDomains)
    newGOlist = getGOfromAnalysis(goAnalysis)
    
    # estimate num sites to sample: numSitesSamp
    nBSL = nRandSitesSim(numsites,5)
    nPrime2 = 0 # Total num genes mapped to figure out nPrime value eventually for estimation
    for BSL in nBSL:
        nPrime2 += len(geneReadSites(BSL, geneL, method))
    nPrime = int(nPrime2 / 5)
    numSitesSamp = int((numMapped * numsites) / nPrime) # Num mapped genes * num sites inputted divided by nPrime 
    
    return simulation(newGOlist, geneL, simN, numSitesSamp, goAnalysis, method, geneClassification, entrezDomains) # GeneL is list of genes with window bounds 

In [112]:
def inANOTB(a,b):
    a_set = set(a) 
    b_set = set(b) 
    return len(list(set(a_set) - set(b_set)))
    

In [113]:
# First run that sets up for the simulation
def adaptiveEnrichmentAnalysis(userSitesL, window, method='TSS'): # userSites must be ordered by chromosome number (List of lists) # Default simN = 10000
    if method == 'TSS':
        geneL = addWindowTSS(window)
        windowDF = addWindowTSS20(window)
    elif method == 'BODY': 
        geneL = addWindowBODY(window)
        windowDF = addWindowBODY20(window)
    else:
        return 'Only TSS or BODY method allowed!'
    
    geneClassification = findSMLdomain(windowDF)
    smallbound = geneClassification[0]
    largebound = geneClassification[1]
    
    mappedGenes = geneReadSites(userSitesL, geneL, method)

    entrezDomains = getEntrezDomain(windowDF)
    smallGenesMAPPED = []# in mapped
    mediumGenesMAPPED = []
    largeGenesMAPPED = []
    for entrez in mappedGenes:
        geneLEN = entrezDomains[entrez]
        if geneLEN < smallbound:
            smallGenesMAPPED.append(entrez)
        elif geneLEN > largebound:
            largeGenesMAPPED.append(entrez)
        else:
            mediumGenesMAPPED.append(entrez)
    
    smallGenesAll = []# in All
    mediumGenesAll = []
    largeGenesAll = []
    
    everygene = []
    for l in windowDF:
        everygene += list(l.entrezid.values)
    for entrez in everygene:
        geneLEN = entrezDomains[entrez]
        if geneLEN < smallbound:
            smallGenesAll.append(entrez)
        elif geneLEN > largebound:
            largeGenesAll.append(entrez)
        else:
            mediumGenesAll.append(entrez)
            
    goAssocGenes = getOntologyID(mappedGenes)
    go2pvals = []
    
    for go in goAssocGenes:
        associated2GO = go2gene[go]
        smallASSOCgo = common_member(smallGenesAll, associated2GO)# in mapped
        mediumASSOCgo = common_member(mediumGenesAll, associated2GO)
        largeASSOCgo = common_member(largeGenesAll, associated2GO)
        
        if len(smallASSOCgo) > 0:
            A = len(common_member(smallASSOCgo, smallGenesMAPPED))
            B = len(smallASSOCgo) - A
            C = inANOTB(smallGenesMAPPED, smallASSOCgo)
            D = len(smallGenesAll) - A - B - C
            smallODDpval = stats.fisher_exact([[A, B], [C, D]])
        else:
            smallODDpval = ('NA','NA')
        
        if len(mediumASSOCgo) > 0:
            E = len(common_member(mediumASSOCgo, mediumGenesMAPPED))
            F = len(mediumASSOCgo) - E
            G = inANOTB(mediumGenesMAPPED, mediumASSOCgo)
            H = len(mediumGenesAll) - E - F - G
            mediumODDpval = stats.fisher_exact([[E, F], [G, H]])
        else:
            mediumODDpval = ('NA','NA')
        
        if len(largeASSOCgo) > 0: 
            I = len(common_member(largeASSOCgo, largeGenesMAPPED))
            J = len(largeASSOCgo) - I
            K = inANOTB(largeGenesMAPPED, largeASSOCgo)
            L = len(largeGenesAll) - I - J - K 
            largeODDpval = stats.fisher_exact([[I, J], [K, L]])
        else:
            largeODDpval = ('NA','NA')

        go2pvals.append([go,smallODDpval, mediumODDpval, largeODDpval])

            
    return go2pvals
    

In [114]:
def adjP(dataSET):
    GOdataSet = dataSET
    goGrp = [[],[],[],[],[],[],[],[],[],[]]
    numAssGrp = [[1], [2], [3], [4], [5, 6], [7, 8], [9, 10, 11, 12], [13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33], [34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 137, 138, 139, 140, 141, 142, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 160, 161, 163, 164, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 182, 183, 185, 187, 188, 189, 190, 191, 193, 194, 196, 198, 200, 201, 202, 203, 204, 205, 206, 208, 210, 211, 212, 213, 214, 216, 217, 218, 219, 221, 225, 226, 229, 232, 234, 235, 236, 238, 239, 240, 242, 244, 249, 250, 251, 252, 254, 262, 263, 264, 265, 268, 272, 273, 278, 280, 282, 285, 290, 295, 297, 301, 307, 308, 310, 311, 312, 313, 315, 324, 329, 330, 334, 343, 344, 347, 354, 360, 362, 364, 366, 368, 371, 374, 375, 385, 387, 395, 397, 405, 409, 411, 413, 420, 424, 427, 431, 439, 441, 457, 458, 467, 474, 481, 483, 491, 496, 502, 504, 505, 524, 529, 537, 540, 547, 548, 552, 565, 578, 605, 610, 611, 667, 670, 689, 703, 706, 812, 816, 859, 927, 960, 973, 980, 1059, 1132, 1134, 1150, 1227, 1304, 1381, 1388, 1456, 1473, 1546, 1778, 1911, 1987, 2163, 2292, 3168, 3637, 4438, 4482, 5026, 5627, 9691]]
    numGO = [5184, 2895, 1743, 1241, 1551, 1018, 1219, 1028, 1020, 1275] # Number of GO terms in each group above.
    
    for go in GOdataSet:
        numAss = len(go2gene[go[0]])
        for i in range(len(numAssGrp)):
            if numAss in numAssGrp[i]:
                goGrp[i].append(go)
    
    pos = 0
    for GOdata in goGrp:
        fishersP = []
#         refabsP = []
        for l in GOdata:
            fishersP.append(l[1][1])
#             refabsP.append(l[1][2])
         
        if len(fishersP) < numGO[pos]:
            numAppend = numGO[pos] - len(fishersP)
            fishersP += [1] * numAppend
#             refabsP += [1] * numAppend
        pos += 1         
        
        reject, fishersPadj, alphacSidak, alphacBonf = padjust(fishersP, method='fdr_bh', is_sorted=False)
#         reject2, refabsPadj, alphacSidak2, alphacBonf2 = padjust(refabsP, method='fdr_bh', is_sorted=False)
        correctedfishers = []
#         correctedrefabs = []
        for i in range(len(GOdata)):
            correctedfishers.append(float(fishersPadj[i]))
#             cohttp://localhost:8890/notebooks/GOfunctionalAnalysis.ipynb#rrectedrefabs.append(float(refabsPadj[i]))
        
        for i in range(len(GOdata)):
            GOdata[i][1].append(correctedfishers[i])
#             GOdata[i][1].append(correctedrefabs[i])
    groupedGO = []
    for i in goGrp:
        for j in i:
            groupedGO.append(j)
    groupedGO = sorted(groupedGO, key = lambda x: x[1][1])
    refabsP = []
    for go in groupedGO:
        refabsP.append(go[1][2])
    reject2, refabsPadj, alphacSidak2, alphacBonf2 = padjust(refabsP, method='fdr_bh', is_sorted=False)
    
    for i in range(len(groupedGO)):
        groupedGO[i][1].append(refabsPadj[i])
    
    return groupedGO # go: [odds, fishersP, refabsP, correctedfishers, correctedrefabs]

In [27]:
# geneAnnotationDF = pd.read_csv('entrez_id/geneAnnotationsDF_Selected_entrezID.csv', sep=',', comment='#', low_memory=False, header=0, names=geneColID)
linSites = pd.read_excel('bindingSites2Test.xlsx', header=0)